In [1]:
import random
import numpy as np
import torch
from PIL import Image
from tqdm import tqdm

seed = 5
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)

def dataset_tensor_to_np(tensor, dtype='image'):
    assert dtype in ['image', 'events', 'seg']
    output = None
    mean_std = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    cityscapes_color_list = [128, 64, 128, 244, 35, 232, 70, 70, 70, 102, 102, 156, 190, 153, 153, 153, 153, 153,
                             250, 170, 30, 220, 220, 0, 107, 142, 35, 152, 251, 152, 70, 130, 180, 220, 20, 60,
                             255, 0, 0, 0, 0, 142, 0, 0, 70, 0, 60, 100, 0, 80, 100, 0, 0, 230, 119, 11, 32]
    if dtype == 'image':
        tensor = tensor.cpu().numpy()
        for index in range(tensor.shape[0]):
            tensor[index, :, :] = tensor[index, :, :] * mean_std[1][index] + mean_std[0][index]
        output = np.uint8(np.transpose(tensor * 255, (1, 2, 0)))  # (0, 1)
    elif dtype == 'events':
        tensor = tensor.cpu().numpy()
        if tensor.shape[0] == 3:
            tensor = np.uint8((tensor + 1) / 2 * 255)
            output = np.transpose(tensor, (1, 2, 0))
        else:
            tensor = np.uint8((tensor[0] + 1) / 2 * 255)
            tensor = np.expand_dims(tensor, axis=0).repeat(3, axis=0)
            output = np.transpose(tensor, (1, 2, 0))
    elif dtype == 'seg':
        if tensor.shape[0] != 1:
            tensor = torch.argmax(tensor, dim=0)
        else:
            tensor = tensor[0]
        tensor = tensor.cpu().numpy()
        tensor = Image.fromarray(tensor.astype(np.uint8)).convert('P')
        tensor.putpalette(cityscapes_color_list)
        output = np.uint8(np.array(tensor.convert('RGB')))
    return output


def tensor_normalize_to_range(tensor, min_val, max_val):
    tensor_min = torch.min(tensor)
    tensor_max = torch.max(tensor)
    tensor = (tensor - tensor_min) / (tensor_max - tensor_min + 1e-8) * (max_val - min_val) + min_val
    return tensor

def get_ic(image_front, image_now, val_range=(0, 255), threshold=0.1, clip_range=0.3):
    image_front = np.asarray(image_front, dtype=np.float32)
    image_front = np.log(image_front / 255 * (val_range[1] - val_range[0]) + val_range[0])
    image_now = np.asarray(image_now, dtype=np.float32)
    image_now = np.log(image_now / 255 * (val_range[1] - val_range[0]) + val_range[0])
    image_change_ = torch.unsqueeze(torch.from_numpy(image_now - image_front), dim=0)
    threshold = (np.log(val_range[1]) - np.log(val_range[0])) * threshold
    clip_range = (np.log(val_range[1]) - np.log(val_range[0])) * clip_range
    mask = (torch.abs(image_change_) <= threshold)
    image_change_[mask] = 0
    image_change_smaller_0 = image_change_.detach().clone()
    image_change_[image_change_ < 0] = 0
    image_change_ = torch.clamp(image_change_, 0, clip_range)
    image_change_ = tensor_normalize_to_range(image_change_, min_val=0, max_val=1)
    image_change_smaller_0[image_change_smaller_0 > 0] = 0
    image_change_smaller_0 = torch.clamp(image_change_smaller_0, -clip_range, 0)
    image_change_smaller_0 = tensor_normalize_to_range(image_change_smaller_0, min_val=-1, max_val=0)
    image_change_ += image_change_smaller_0
    return image_change_


def get_image_change_from_pil(pil_image, width, height, data_type, shift_pixel=4, val_range=None,
                              _threshold=None, _clip_range=None):
    assert data_type in {'day', 'night', 'new_day'}

    if val_range is not None:
        val_range = val_range
    if _threshold is not None:
        threshold = _threshold
    if _clip_range is not None:
        clip_range = _clip_range

    inputs_gray = np.array(pil_image.convert('L'))  # (480, 640)
    inputs_left = np.concatenate((inputs_gray[:, shift_pixel:], inputs_gray[:, width-shift_pixel:]), axis=1)
    inputs_right = np.concatenate((inputs_gray[:, :shift_pixel], inputs_gray[:, :width-shift_pixel]), axis=1)
    inputs_up = np.concatenate((inputs_gray[shift_pixel:, :], inputs_gray[height-shift_pixel:, :]), axis=0)
    inputs_down = np.concatenate((inputs_gray[:shift_pixel, :], inputs_gray[:height-shift_pixel, :]), axis=0)
    # image_change_1 = get_ic(inputs_gray, inputs_right, val_range=val_range, threshold=threshold, clip_range=clip_range)
    # image_change_2 = get_ic(inputs_gray, inputs_down, val_range=val_range, threshold=threshold, clip_range=clip_range)
    image_change_1 = get_ic(inputs_gray, inputs_up, val_range=val_range, threshold=threshold, clip_range=clip_range)
    image_change_2 = get_ic(inputs_gray, inputs_left, val_range=val_range, threshold=threshold, clip_range=clip_range)

    # image_change_3 = get_ic(inputs_gray, inputs_down, val_range=val_range, threshold=threshold, clip_range=clip_range)
    # image_change_4 = get_ic(inputs_gray, inputs_right, val_range=val_range, threshold=threshold, clip_range=clip_range)

    image_avg = image_change_1 / 2 + image_change_2 / 2
    # image_avg = image_change_1 / 4 + image_change_2 / 4 + image_change_3 / 4 + image_change_4 / 4
    return image_avg  # tensor -1 ~ +1

print('done!')

done!


In [6]:
import torch
import torch.nn as nn
import numpy as np
from PIL import Image


kirsch_kernels = [
    torch.tensor([[ 5,  5,  5],
                  [-3,  0, -3],
                  [-3, -3, -3]], dtype=torch.float),
    torch.tensor([[-3,  5,  5],
                  [-3,  0,  5],
                  [-3, -3, -3]], dtype=torch.float),
    torch.tensor([[-3, -3,  5],
                  [-3,  0,  5],
                  [-3, -3,  5]], dtype=torch.float),
    torch.tensor([[-3, -3, -3],
                  [-3,  0,  5],
                  [-3,  5,  5]], dtype=torch.float),
    torch.tensor([[-3, -3, -3],
                  [-3,  0, -3],
                  [ 5,  5,  5]], dtype=torch.float),
    torch.tensor([[-3, -3, -3],
                  [ 5,  0, -3],
                  [ 5,  5, -3]], dtype=torch.float),
    torch.tensor([[ 5, -3, -3],
                  [ 5,  0, -3],
                  [ 5, -3, -3]], dtype=torch.float),
    torch.tensor([[ 5,  5, -3],
                  [ 5,  0, -3],
                  [-3, -3, -3]], dtype=torch.float)
]

rho_kernels = [
    torch.tensor([[ 3, -1],
                  [-1, -1]], dtype=torch.float),
    torch.tensor([[-1,  3],
                  [-1, -1]], dtype=torch.float),
    torch.tensor([[-1, -1],
                  [-1,  3]], dtype=torch.float),
    torch.tensor([[-1, -1],
                  [ 3, -1]], dtype=torch.float),
]

# rho_kernels = [
#     torch.tensor([[ 1,  0],
#                   [ 0, -1]], dtype=torch.float),
#     torch.tensor([[ 0,  1],
#                   [-1,  0]], dtype=torch.float),
#     torch.tensor([[-1,  0],
#                   [ 0,  1]], dtype=torch.float),
#     torch.tensor([[ 0, -1],
#                   [ 1,  0]], dtype=torch.float),
# ]

diff_kernels = [
    # torch.tensor([[ 1, 0],
    #               [ 0, -1]], dtype=torch.float),
    torch.tensor([[ 3, -1],
                  [-1, -1]], dtype=torch.float),
]

class Kirsch(nn.Module):
    def __init__(self):
        super(Kirsch, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, kernel_size=3, padding=1, bias=False, padding_mode='reflect')
        with torch.no_grad():
            for i, kernel in enumerate(kirsch_kernels):
                self.conv1.weight[i].copy_(kernel.unsqueeze(0))

    def forward(self, x):
        return self.conv1(x)

class Rho(nn.Module):
    def __init__(self):
        super(Rho, self).__init__()
        self.conv1 = nn.Conv2d(1, 4, kernel_size=2, padding=1, bias=False, padding_mode='reflect')
        for i, kernel in enumerate(rho_kernels):
            self.conv1.weight[i].copy_(kernel.unsqueeze(0))

    @torch.no_grad()
    def forward(self, x):
        return self.conv1(x)


class Diff(nn.Module):
    def __init__(self):
        super(Diff, self).__init__()
        self.conv1 = nn.Conv2d(1, 1, kernel_size=2, padding=1, bias=False, padding_mode='reflect')
        for i, kernel in enumerate(diff_kernels):
            self.conv1.weight[i].copy_(kernel.unsqueeze(0))

    @torch.no_grad()
    def forward(self, x):
        return self.conv1(x)

def print_tensor_shape_min_max(tensor):
    print('shape: {}, min: {}, max: {}'.format(tensor.shape, torch.min(tensor), torch.max(tensor)))


def load_image(image_path):
    img = Image.open(image_path).convert('RGB')
    gray = np.array(img.convert('L'))
    gray_tensor = torch.from_numpy(gray).unsqueeze(0).unsqueeze(0).float() / 255.0  # 0~1
    return gray_tensor, img


@torch.no_grad()
def conv_operation_on_image(gray_tensor, kernel_type, min_parm=0.2, max_parm=0.999):
    assert kernel_type in {'diff', 'rho', 'kirsch'}
    if kernel_type == 'diff':
        operation = Diff()
    elif kernel_type == 'rho':
        operation = Rho()
    else:
        operation = Kirsch()
    edges_tensor = operation(torch.log((gray_tensor + 1e-2)))[:, :, :-1, :-1]
    if kernel_type in {'rho', 'kirsch'}:
        edges_tensor = torch.max(edges_tensor, dim=1, keepdim=True)[0]
    print_tensor_shape_min_max(edges_tensor)
    min_clip_thres = min_parm
    if kernel_type == 'diff':
        max_clip_thres = torch.quantile(edges_tensor[edges_tensor > 0], max_parm)  # edges_tensor[edges_tensor > 0]  torch.abs(edges_tensor)
        print(max_clip_thres)
        edges_tensor[torch.abs(edges_tensor) < min_clip_thres] = 0
        edges = torch.clamp(edges_tensor, min=-max_clip_thres, max=max_clip_thres) / max_clip_thres
        edges = (edges + 1) / 2
    else:
        max_clip_thres = torch.quantile(edges_tensor, max_parm)
        edges = torch.clamp(edges_tensor, min=min_clip_thres, max=max_clip_thres)
        edges = (edges - min_clip_thres) / (max_clip_thres - min_clip_thres)
    return edges

# from dsec_dataset import DSECDataset
# dataset = DSECDataset(root_path='G:/DSEC_Dataset/', events_bins=20, events_clip_range=0.95, crop_size=(400, 400),
#                       sequences_num=1, events_step=1, ignore_down_pixel=20, test_flag=False, test_seg=True)


# image_path = r'G:\DSEC_Dataset\train_wrap_images\zurich_city_06_a\000000.png'
image_path = r'E:\DSEC_Dataset\train_wrap_images\zurich_city_05_b\000016.png'
# image_path = r'D:\研究生\Python\HANet\cityscapes\leftImg8bit\train\cologne\cologne_000037_000019_leftImg8bit.png'
kernel_type = 'diff'  # diff rho kirsch
min_parm = 0
max_parm = 0.999

gray_tensor, img = load_image(image_path)
edges = conv_operation_on_image(gray_tensor, kernel_type, min_parm, max_parm)

# mse_mean_loss = nn.MSELoss(reduction='mean')
# mse_none_loss = nn.MSELoss(reduction='none')
# edges_zeros = torch.zeros_like(edges)
# edges_ones = torch.zeros_like(edges) + 1
#
# edges_max_weight = 50
#
# base_num = 3
# edges_weight = 1 / (base_num - 1) * (base_num * (1 - edges_max_weight) * torch.pow(base_num, -edges) + base_num * edges_max_weight - 1)
# zeros_loss = mse_none_loss(edges, edges_zeros) * edges_weight
# zeros_loss = zeros_loss.sum() / edges.numel()
#
# ones_loss = mse_none_loss(edges, edges_ones) * edges_weight
# ones_loss = ones_loss.sum() / edges.numel()
#
# print(torch.mean(edges))
# print(zeros_loss, ones_loss)

edges = np.uint8(edges[0, 0][:, :, None].repeat(1, 1, 3).numpy() * 255)
output_show_image = np.concatenate((np.array(img), edges), axis=1)
Image.fromarray(output_show_image).save(r'D:\研究生\总结\2023.5.6总结\2.png')
Image.fromarray(output_show_image).show()

shape: torch.Size([1, 1, 480, 640]), min: -4.4760637283325195, max: 5.348857879638672
tensor(2.8221)


In [ ]:
from PIL import Image
import numpy as np
import torchvision.transforms as standard_transforms
import torch
import torch.nn.functional as F

# mean_std = ([0.5, 0.5, 0.5], [0.229, 0.224, 0.225])

mean_std = ([127, 127, 127], [58.395, 57.12, 57.375])
img_norm_cfg = dict(mean=[123.675, 116.28, 103.53], std=[58.395, 57.12, 57.375], to_rgb=True)
pil_to_tensor_transform = standard_transforms.Compose([standard_transforms.PILToTensor()])
HorizontalFlip = standard_transforms.RandomHorizontalFlip(p=1)

image_path = 'E:/dsec_dataset/train_edges/zurich_city_01_a/000024.png'
edges_pil = Image.open(image_path).convert('RGB')

edges_torch = pil_to_tensor_transform(edges_pil)
edges_torch = edges_torch.to(torch.float32)
edges_torch = standard_transforms.functional.normalize(edges_torch, mean_std[0], mean_std[1])

edges_torch_flip = HorizontalFlip(edges_pil)

print(type(edges_torch_flip))
print(edges_torch.shape, torch.min(edges_torch), torch.max(edges_torch), edges_torch.dtype)


In [ ]:
a = [1, 2]
print(tuple(a))

In [1]:
a = [1, 2]
print(tuple(a))

(1, 2)
